In [37]:
import os 
import cv2 
import matplotlib.pyplot as plt 
import numpy as np 
import dlib

In [ ]:
my_image_path = 'aiffel/camera_sticker/images/MyFace.png' 
img_bgr = cv2.imread(my_image_path)   
img_show = img_bgr.copy()      # 출력용 이미지를 따로 보관
plt.imshow(img_bgr)
plt.show() 

In [ ]:
img_rgb = cv2.cvtColor( img_bgr, cv2.COLOR_BGR2RGB )
plt.imshow( img_rgb ) 
plt.show() 

In [27]:
# 기본 얼굴 감지기
detector_hog = dlib.get_frontal_face_detector() 

In [ ]:
# 얼굴의 bounding box를 추출
dlib_rects = detector_hog(img_rgb, 1)

In [ ]:
# 찾은 얼굴 영역의 좌표
for  dlib_rect  in  dlib_rects: 
    l = dlib_rect.left()
    t = dlib_rect.top()
    r = dlib_rect.right() 
    b = dlib_rect.bottom() 
    cv2.rectangle( img_show, (l,t), (r,b), (0,255,0), 2, lineType=cv2.LINE_AA ) 
     # 시작점 좌표(l,t)와 종료점 좌표(r,b)로 직사각형을 그림

img_show_rgb =  cv2.cvtColor( img_show, cv2.COLOR_BGR2RGB )
plt.imshow( img_show_rgb )
plt.show()

In [ ]:
model_path = 'aiffel/camera_sticker/models/shape_predictor_68_face_landmarks.dat' 
landmark_predictor = dlib.shape_predictor(model_path)

list_landmarks = []
 for dlib_rect in dlib_rects:
      # 모든 landmark의 위치정보를 points 변수에 저장함.
    points = landmark_predictor( img_rgb, dlib_rect )
      # 각각의 landmark 위치정보를 (x,y) 형태로 변환하여 리스트로 저장함.
    list_points = list( map( lambda p: (p.x, p.y), points.parts()))  # parts() 함수로 개별 위치에 접근
    list_landmarks.append( list_points )
print(len(list_landmarks[0])) # 각 원소는 68개의 랜드마크 위치가 나열된 리스트임.

In [ ]:
# 랜드마크를 영상에 출력
for  landmark  in  list_landmarks:
    for  point  in  landmark:
        cv2.circle( img_show, point, 2, (0, 255, 255), -1)
            # img_show 이미지 위 각각의 point에 크기가 2이고 (0, 255, 255)색으로 내부가 채워진(-1) 원을 그림.
img_show_rgb = cv2.cvtColor( img_show, cv2.COLOR_BGR2RGB )
plt.imshow( img_show_rgb )
plt.show()

In [ ]:
for  dlib_rect, landmark  in  zip( dlib_rects, list_landmarks ):
    print(landmark[30]) # 코의 index
    x = landmark[30][0] # 이미지에서 코 부위의 x값
    y = landmark[30][1] - dlib_rect.height() // 2  
     # 이미지에서 코 부위의 y값 - 얼굴 영역의 세로를 차지하는 픽셀의 수
    w = h = dlib_rect.width() # 얼굴 영역의 가로를 차지하는 픽셀의 수 
    print (f'(x,y) : ({x},{y})')
    print (f'(w,h) : ({w},{h})')

In [ ]:
# 준비해 둔 스티커 이미지를 읽어서 적용
sticker_path = os.getenv('HOME') + '/aiffel/camera_sticker/images/cat-whiskers.png' 
img_sticker = cv2.imread( sticker_path ) #  image객체 행렬을 반환함.
img_sticker = cv2.resize( img_sticker, (w,h) ) # 스티커 이미지 조정

In [ ]:
refined_x = x - w // 2  
refined_y = y - h    
# refined_x, refined_y값에서 스티커 이미지가 시작됨.

In [ ]:
print (f'(x,y) : ({refined_x},{refined_y})')  # 이미지의 범위를 초과하면 음수가 발생함.
# opencv는 음수 인덱스에 접근 불가하므로 스티커 이미지를 잘라줘야함.

In [ ]:
# 음수값만큼 스티커 이미지를 자름.
if refined_x < 0: 
    img_sticker = img_sticker[ :, -refined_x: ]
    refined_x = 0

In [ ]:
# 스티커 이미지가 시작할 y좌표값 조정
if refined_y < 0:
    img_sticker = img_sticker[ -refined_y:, : ] 
    refined_y = 0
print (f'(x,y) : ({refined_x},{refined_y})')

In [ ]:
# sticker_area는 원본 이미지에서 스티커를 적용할 위치를 미리 잘라낸 이미지임.
sticker_area = img_show[ refined_y : refined_y + img_sticker.shape[0],  refined_x : refined_x + img_sticker.shape[1]]

In [ ]:
# 스티커 이미지에서 0이 나오는 부분은 이미지가 없는 부분임.  ->  sticker_area 적용
img_show[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]] 
  = np.where( img_sticker==0, sticker_area, img_sticker ).astype(np.uint8)

In [ ]:
# 얼굴 영역과 랜드마크를 미리 적용해놓은 img_show에 스티커 이미지를 덧붙인 이미지가 나옴.
plt.imshow( cv2.cvtColor( img_show, cv2.COLOR_BGR2RGB ))
plt.show()

In [ ]:
 # bounding box와 landmark를 제거하고 최종 결과만 출력
sticker_area = img_bgr[ refined_y : refined_y  + img_sticker.shape[0],  refined_x : refined_x + img_sticker.shape[1]]

In [ ]:
 # img_bgr은 rgb로만 적용해놓은 원본 이미지임.
img_bgr[refined_y : refined_y  + img_sticker.shape[0],  refined_x : refined_x + img_sticker.shape[1]] = np.where( img_sticker==0, sticker_area, img_sticker).astype(np.uint8)
plt.imshow( cv2.cvtColor( img_bgr, cv2.COLOR_BGR2RGB)) 
plt.show()